In [173]:
import pandas as pd
import math
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# read in data predictions for BTC

In [174]:
predictions_df = pd.read_csv("training_configs/btc_all_predictions.csv", parse_dates=True)
true_price_df = pd.read_csv("../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv")

In [175]:
predictions_df.head()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
0,2010-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010-01-09,0.0
1,2019-04-02,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,2019-04-09,0.0
2,2019-04-03,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,2019-04-10,0.0
3,2019-04-04,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,2019-04-11,0.0
4,2019-04-05,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,2019-04-12,0.0


In [176]:
predictions_df.date.max()

'2020-02-16'

In [177]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 9 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   date                                                    322 non-null    object 
 1   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  322 non-null    float64
 2   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     322 non-null    float64
 3   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  322 non-null    float64
 4   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     322 non-null    float64
 5   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  322 non-null    float64
 6   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     322 non-null    float64
 7   date_prediction_for                                     322 non-null    object 
 8   test_model_lookback_1                    

In [178]:
true_close_df = true_price_df[['date','close']]

### Map the predictions for date to the true price date

In [179]:
predictions_df.date_prediction_for

0       2010-01-09
1       2019-04-09
2       2019-04-10
3       2019-04-11
4       2019-04-12
          ...     
317     2020-02-19
318     2020-02-20
319     2020-02-21
320     2020-02-22
321     2020-02-23
Name: date_prediction_for, Length: 322, dtype: object

In [230]:
merged_df = pd.merge(predictions_df, true_close_df, left_on='date_prediction_for',
                     right_on ='date', suffixes=['_pred','_true'])

In [231]:
merged_df['day'] = [t.day for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['month'] =  [t.month for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['quarter'] = [t.quarter for t in pd.to_datetime(merged_df.date_prediction_for)]

In [232]:
merged_df.drop(['date_pred', 'date_true', 'date_prediction_for'], inplace=True, axis=1)

In [233]:
merged_df.tail()

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter
316,10678.604442,9321.594630,10273.572624,9467.310152,9270.379064,9281.113845,0.0,9633.39,19,2,1
317,10199.084654,10374.727670,9982.991802,9541.344697,9618.900470,9093.893937,0.0,9608.48,20,2,1
318,9789.181893,10262.505174,9318.772646,9708.788008,8997.713868,9983.222021,0.0,9686.44,21,2,1
319,9950.963327,10107.269446,10088.766177,9478.738641,10524.815887,9626.286500,0.0,9663.18,22,2,1
320,10574.311071,10030.417329,10466.793448,10224.888552,10242.701565,9669.070675,0.0,9924.52,23,2,1


In [257]:
merged_df.describe()

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter
count,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.0,321.000000,321.000000,321.00000,321.000000
mean,8922.425006,9038.582519,9013.820224,9296.475205,9110.479439,8828.131398,0.0,8733.008162,15.813084,7.00000,2.691589
std,1984.173699,4964.035630,2233.696650,4758.362516,2416.333616,5015.274501,0.0,1697.884398,8.690653,3.41321,1.061348
min,4546.248123,139.877840,3666.543647,-289.448271,3261.251466,-280.599843,0.0,5064.490000,1.000000,1.00000,1.000000
25%,7514.535066,5765.344232,7610.308052,6822.783546,7677.698838,6626.769284,0.0,7564.350000,9.000000,5.00000,2.000000
50%,8945.097083,9012.645122,8966.104576,9044.588106,8884.057393,8855.334264,0.0,8680.880000,16.000000,7.00000,3.000000
75%,10368.821781,11386.225395,10588.985354,11575.738359,11130.186430,10682.674332,0.0,10115.980000,23.000000,10.00000,4.000000
max,13331.237899,23654.633486,14332.008433,23157.885335,14228.875403,23173.074402,0.0,13016.230000,31.000000,12.00000,4.000000


### train test split

In [234]:
split_pct = .95
l_merged = len(merged_df)
merged_df_train, merged_df_test = merged_df.iloc[:int(split_pct*l_merged),:],  merged_df.iloc[int(l_merged*split_pct):,:]
                                                                                                  

In [235]:
merged_df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 0 to 303
Data columns (total 11 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  304 non-null    float64
 1   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     304 non-null    float64
 2   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  304 non-null    float64
 3   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     304 non-null    float64
 4   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  304 non-null    float64
 5   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     304 non-null    float64
 6   test_model_lookback_1                                   304 non-null    float64
 7   close                                                   304 non-null    float64
 8   day                                     

In [236]:
merged_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 304 to 320
Data columns (total 11 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  17 non-null     float64
 1   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     17 non-null     float64
 2   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  17 non-null     float64
 3   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     17 non-null     float64
 4   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  17 non-null     float64
 5   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     17 non-null     float64
 6   test_model_lookback_1                                   17 non-null     float64
 7   close                                                   17 non-null     float64
 8   day                                    

In [237]:
merged_df_train.iloc[:, ~merged_df.columns.isin(['close'])]

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,day,month,quarter
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,9,4,2
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,10,4,2
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,11,4,2
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,12,4,2
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,13,4,2
...,...,...,...,...,...,...,...,...,...,...
299,9080.042214,8234.266424,8458.096843,8663.166443,8713.979386,8117.061217,0.0,2,2,1
300,8816.222948,8801.989284,8837.710458,8124.499700,8145.578109,8188.955274,0.0,3,2,1
301,8794.896171,8186.986526,9764.729837,8460.526527,8067.736909,8323.349055,0.0,4,2,1
302,9016.097942,8836.810967,8525.258736,8871.649255,8407.724381,7999.043911,0.0,5,2,1


In [238]:


merged_df_x_train, merged_df_y_train = merged_df_train.iloc[:, ~merged_df_train.columns.isin(['close'])], merged_df_train.iloc[:, merged_df_train.columns.isin(['close'])]
merged_df_x_test, merged_df_y_test =  merged_df_test.iloc[:, ~merged_df_test.columns.isin(['close'])], merged_df_test.iloc[:, merged_df_test.columns.isin(['close'])]
                                                                                                                                                 

In [239]:
merged_df_x_train

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,day,month,quarter
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,9,4,2
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,10,4,2
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,11,4,2
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,12,4,2
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,13,4,2
...,...,...,...,...,...,...,...,...,...,...
299,9080.042214,8234.266424,8458.096843,8663.166443,8713.979386,8117.061217,0.0,2,2,1
300,8816.222948,8801.989284,8837.710458,8124.499700,8145.578109,8188.955274,0.0,3,2,1
301,8794.896171,8186.986526,9764.729837,8460.526527,8067.736909,8323.349055,0.0,4,2,1
302,9016.097942,8836.810967,8525.258736,8871.649255,8407.724381,7999.043911,0.0,5,2,1


In [240]:
merged_df_y_train

,close
0,5204.96
1,5324.55
2,5064.49
3,5089.54
4,5096.59
...,...
299,9344.37
300,9293.52
301,9180.96
302,9613.42


### performance, rmse , on average prediction

In [241]:
average_predictions_train = np.mean(merged_df_x_train.iloc[:,:6], axis=1)
average_predictions_test = np.mean(merged_df_x_test.iloc[:,:6], axis=1)

In [242]:
average_predictions_train[:10]

0    2442.651505
1    2572.342291
2    2619.974055
3    2587.341908
4    2852.544153
5    2721.518844
6    3153.856190
7    2727.367343
8    3387.347636
9    2872.964842
dtype: float64

In [243]:
merged_df_y_train.head(10)

,close
0,5204.96
1,5324.55
2,5064.49
3,5089.54
4,5096.59
5,5167.72
6,5067.11
7,5235.56
8,5251.94
9,5298.39


In [244]:
average_rmse_train = sqrt(mean_squared_error(average_predictions_train, merged_df_y_train))
average_rmse_test = sqrt(mean_squared_error(average_predictions_test, merged_df_y_test))

In [245]:
print(f" The average training rmse is {average_rmse_train}")
print(f" The average testing rmse is {average_rmse_test}")

 The average training rmse is 2138.8420481268163
 The average testing rmse is 809.7787104960843


# train linear regression on ALL predictions

In [246]:
lregr = LinearRegression()

In [247]:
lregr.fit(merged_df_x_train, merged_df_y_train)

LinearRegression()

In [248]:
train_predictions_lrgr = lregr.predict(merged_df_x_train)

In [249]:
test_predictions_lrgr = lregr.predict(merged_df_x_test)

In [260]:
test_predictions_lrgr[:15]


array([[ 9534.08472944],
       [ 9618.37175695],
       [ 9265.18484903],
       [ 9515.63188658],
       [ 9393.57980591],
       [ 9497.21239144],
       [ 9601.88923898],
       [ 9714.60526263],
       [ 9533.91941713],
       [ 9572.33841449],
       [10102.68749555],
       [ 9746.67485573],
       [10074.92759153],
       [10008.85733741],
       [ 9643.5488235 ]])

In [261]:
merged_df_y_test.head(15)

,close
304,9795.94
305,9865.12
306,10116.67
307,9856.61
308,10208.24
309,10326.05
310,10214.38
311,10312.12
312,9889.42
313,9934.43


In [252]:
average_rmse_train_lrgr = sqrt(mean_squared_error(train_predictions_lrgr, merged_df_y_train))
average_rmse_test_lrgr = sqrt(mean_squared_error(test_predictions_lrgr, merged_df_y_test))

In [253]:
print(f" The average training rmse with linear stacking is {average_rmse_train_lrgr}")
print(f" The average testing rmse with linear stacking is {average_rmse_test_lrgr}")

 The average training rmse with linear stacking is 875.3816880537928
 The average testing rmse with linear stacking is 508.7666502563874




#### feature importance

In [254]:
lregr.coef_.ravel()

array([ 2.25847223e-01,  1.25276099e-02,  1.74230192e-01,  3.30826548e-02,
        2.37544829e-01, -7.43483871e-03,  1.79056769e-12, -6.96799615e+00,
        1.43361439e+02, -8.46600342e+02])

In [255]:
lregr.coef_.ravel()[np.argsort(lregr.coef_.ravel())[::-1]]

array([ 1.43361439e+02,  2.37544829e-01,  2.25847223e-01,  1.74230192e-01,
        3.30826548e-02,  1.25276099e-02,  1.79056769e-12, -7.43483871e-03,
       -6.96799615e+00, -8.46600342e+02])

In [256]:
merged_df_x_train.columns[np.argsort(lregr.coef_)[::-1]]

array([['quarter', 'day',
        'tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
        'test_model_lookback_1',
        'tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
        'tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
        'month']], dtype=object)